<a href="https://colab.research.google.com/github/rmanzi13/Consultor_Belleza/blob/main/Surprise_Filtrado_Colaborativo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install scikit-surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 4.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3163693 sha256=6c41fc790dcf065fe1b16f93356edcec36cd8610e736177b5702e15d699b7be3
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


In [2]:
import pandas as pd

In [3]:
df_data = pd.read_csv('df_cosmetics_form.csv', low_memory=False)

In [4]:
df_data.head(2)

,Combination,Name_Group_Num,Brand_Group_Num,Name,Brand,Price_Category,Rank_Num,Label_Num,Textura_Piel_Num,Piel_Brillante_Num,Tono_Piel_Num,Granitos_Imperfecciones_Num,Enrojece_Facilidad_Num,Irritacion_Piel_Num,Tirantez_Sequedad_Num,Poros_Abiertos_Num,Alergia_Cosmeticos_Num,Tipo_Piel_Num
0,1.0,1.0,1.0,Crème de la Mer,LA MER,4.0,5.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
1,1.0,1.0,1.0,Facial Treatment Essence,SK-II,4.0,5.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [5]:
print(df_data.isnull().sum())

Combination                        0
Name_Group_Num                     0
Brand_Group_Num                    0
Name                           59049
Brand                          59049
Price_Category                     0
Rank_Num                           0
Label_Num                          0
Textura_Piel_Num                   0
Piel_Brillante_Num                 0
Tono_Piel_Num                      0
Granitos_Imperfecciones_Num        0
Enrojece_Facilidad_Num             0
Irritacion_Piel_Num                0
Tirantez_Sequedad_Num              0
Poros_Abiertos_Num                 0
Alergia_Cosmeticos_Num             0
Tipo_Piel_Num                      0
dtype: int64


### Guardo copia antes de eliminar columnas 3 y 4

In [6]:
# Hacer una copia del DataFrame
df_copia = df_data.copy()

In [7]:
# Eliminar las columnas 3 y 4 del DataFrame original
df_data = df_data.drop(columns=['Name', 'Brand'])

In [8]:
df_data.head(1)

,Combination,Name_Group_Num,Brand_Group_Num,Price_Category,Rank_Num,Label_Num,Textura_Piel_Num,Piel_Brillante_Num,Tono_Piel_Num,Granitos_Imperfecciones_Num,Enrojece_Facilidad_Num,Irritacion_Piel_Num,Tirantez_Sequedad_Num,Poros_Abiertos_Num,Alergia_Cosmeticos_Num,Tipo_Piel_Num
0,1.0,1.0,1.0,4.0,5.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0


In [9]:
# Convierto datas a enteros
df_data = df_data.astype(int)

In [10]:
df_data.head(2)

,Combination,Name_Group_Num,Brand_Group_Num,Price_Category,Rank_Num,Label_Num,Textura_Piel_Num,Piel_Brillante_Num,Tono_Piel_Num,Granitos_Imperfecciones_Num,Enrojece_Facilidad_Num,Irritacion_Piel_Num,Tirantez_Sequedad_Num,Poros_Abiertos_Num,Alergia_Cosmeticos_Num,Tipo_Piel_Num
0,1,1,1,4,5,1,2,2,2,2,2,2,2,2,2,2
1,1,1,1,4,5,1,2,2,2,2,2,2,2,2,2,2


In [11]:
from surprise import Dataset,Reader
from surprise.model_selection import train_test_split
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import GridSearchCV

In [15]:
# Defino un objeto Reader
reader = Reader(rating_scale=(1, 14))

In [16]:
# Defino los parámetros que deseas ajustar
param_grid = {'n_factors': [50, 100, 200],
              'n_epochs': [10, 20, 30],
              'lr_all': [0.002, 0.005, 0.01],
              'reg_all': [0.02, 0.1, 0.2]}

In [17]:
# Creo un objeto SVD
svd = SVD()

In [21]:
# Creo un objeto GridSearchCV
grid_search = GridSearchCV(SVD, param_grid, measures=['rmse'], cv=3, n_jobs=-1)

In [22]:
data = Dataset.load_from_df(df_data[['Name_Group_Num', 'Tipo_Piel_Num', 'Name_Group_Num']], reader)

In [23]:
# Realizar la búsqueda de cuadrícula y ajustar el modelo automáticamente
grid_search.fit(data)

In [26]:
# Imprimir los mejores resultados obtenidos
print("Mejores hiperparámetros:")
print(grid_search.best_params['rmse'])
print("Mejor RMSE obtenido:")
print(grid_search.best_score['rmse'])

Mejores hiperparámetros:
{'n_factors': 50, 'n_epochs': 10, 'lr_all': 0.002, 'reg_all': 0.02}
Mejor RMSE obtenido:
0.08637580434368437


In [28]:
# Divido el conjunto de datos en entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.25)

In [29]:
# Obtengo los mejores hiperparámetros
best_params = {'n_factors': 50, 'n_epochs': 30, 'lr_all': 0.002, 'reg_all': 0.2}

In [30]:
# Inicializo un nuevo modelo SVD con los mejores hiperparámetros
model = SVD(n_factors=best_params['n_factors'],
            n_epochs=best_params['n_epochs'],
            lr_all=best_params['lr_all'],
            reg_all=best_params['reg_all'])

In [31]:
# Entreno el modelo en el conjunto de entrenamiento
model.fit(trainset)

In [32]:
# Realizopredicciones en el conjunto de prueba
predictions = model.test(testset)

In [33]:
# Evaluo la precisión del modelo con el nuevo conjunto de hiperparámetros
accuracy.rmse(predictions)

RMSE: 0.1055


0.10550947007096984

In [34]:
 # Elegir un usuario ficticio (puedes cambiar el ID de usuario según tus datos)
user_id_ficticio = 1

In [35]:
# Hacemos recomendaciones para el usuario ficticio
recommendations = []
for item_id in df_data['Name_Group_Num'].unique():
    pred = model.predict(user_id_ficticio, item_id)
    recommendations.append((item_id, pred.est))

In [36]:
# Ordeno las recomendaciones por puntaje
recommendations = sorted(recommendations, key=lambda x: x[1], reverse=True)

In [37]:
# Imprimo las 5 principales recomendaciones
top_recommendations = recommendations[:5]
for item_id, score in top_recommendations:
    print(f"Nombre del Producto: {item_id}, Puntaje: {score}")

Nombre del Producto: 3, Puntaje: 2.528254449537919
Nombre del Producto: 4, Puntaje: 2.528250284504978
Nombre del Producto: 1, Puntaje: 2.5282494229219217
Nombre del Producto: 5, Puntaje: 2.5269621547898806
Nombre del Producto: 6, Puntaje: 2.5269621547898806


### Buscaremos los nombres correspondientes de los Productos

In [38]:
# Creo un diccionario que mapee los números de productos con sus nombres correspondientes
num_to_name_mapping = df_copia.set_index('Name_Group_Num')['Name'].to_dict()

In [39]:
# Hacemos recomendaciones para el usuario ficticio
recommendations_with_names = []
for item_id, score in top_recommendations:
    name = num_to_name_mapping.get(item_id, f"Producto {item_id}")  # En caso de que no se encuentre el nombre
    recommendations_with_names.append((name, score))

In [40]:
# Imprimo las 5 principales recomendaciones con nombres
for name, score in recommendations_with_names:
    print(f"Nombre del Producto: {name}, Puntaje: {score}")

Nombre del Producto: T.L.C. Framboos™ Glycolic Night Serum, Puntaje: 2.528254449537919
Nombre del Producto: Crème Radiance Gentle Cleansing Creamy-Foam Cleanser, Puntaje: 2.528250284504978
Nombre del Producto: Renewed Hope in A Jar Refreshing & Refining Moisturizer, Puntaje: 2.5282494229219217
Nombre del Producto: Cleansing Gel Oil Enriched, Puntaje: 2.5269621547898806
Nombre del Producto: Powerful-Strength Line-Reducing Concentrate 12.5% Vitamin C, Puntaje: 2.5269621547898806
